In [ ]:
import numpy as np
import json
import pandas as pd
from src.network import Network_custom
import os

Step 1) Open the taget spider web with tension

In [ ]:
with open(os.path.join('DATA', "public_spider_network.json"), "r") as f:
    data = json.load(f)
net = Network_custom.direct(np.array(data["vertices"]), data["edges"], data["q"], data["fixed"])
net.dir = np.ones(len(net.edges))
net.dir[::2] = -1

Step 2) Determine goal edge lengths l0, and optimize vertices

In [ ]:
file_path = os.path.join('DATA', 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path, A_scale = 1.0)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic') # In the paper l_scalar is called 's'

net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 100000, "damping": .1})
net.optimize_vertices()

Animate the Relaxation like this

In [ ]:
%matplotlib qt
ani = net.ShapeOptimizer.animate_optimization()

Step 3) Scale the web with s (l_scalar)

In [ ]:
reference_point = [0,0,0] 
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs=False) # If you don't provide a scalar, it will use network.l_scalar automatically. Note, that in this particular case, there are no leafs, so the last argument is not necessary.

#  You can plot the scaled network like this
# net.net_plot(color=True, plot_type='scaled')

# Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
xyz = net.arc_points(n = 9) 

net.net_plot(color=True, plot_type='arcs', elables=False)
net.l_scalar